In [ ]:
# | default_exp _testing.test_utils

In [ ]:
# | export

import asyncio
import hashlib
import platform
import shlex
import signal
import subprocess  # nosec
import unittest
import unittest.mock
from contextlib import contextmanager
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import *

import asyncer
from IPython.display import IFrame

from fastkafka._application.app import FastKafka
from fastkafka._components._subprocess import terminate_asyncio_process
from fastkafka._components.helpers import _import_from_string, change_dir
from fastkafka._components.logger import get_logger

In [ ]:
import time
from inspect import signature

import anyio
import nest_asyncio
import pytest
from nbdev_mkdocs.docstring import run_examples_from_docstring
from tqdm.notebook import tqdm, trange

from fastkafka._components.logger import suppress_timestamps
from fastkafka._helpers import consumes_messages, produce_messages
from fastkafka._testing.apache_kafka_broker import ApacheKafkaBroker

In [ ]:
# | notest

# allows async calls in notebooks

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def nb_safe_seed(s: str) -> Callable[[int], int]:
    """Gets a unique seed function for a notebook

    Params:
        s: name of the notebook used to initialize the seed function

    Returns:
        A unique seed function
    """
    init_seed = int(hashlib.sha256(s.encode("utf-8")).hexdigest(), 16) % (10**8)

    def _get_seed(x: int = 0, *, init_seed: int = init_seed) -> int:
        return init_seed + x

    return _get_seed

In [ ]:
seed = nb_safe_seed("999_test_utils")

assert seed() == seed(0)
assert seed() + 1 == seed(1)

In [ ]:
# | export


@contextmanager
def mock_AIOKafkaProducer_send() -> Generator[unittest.mock.Mock, None, None]:
    """Mocks **send** method of **AIOKafkaProducer**"""
    with unittest.mock.patch("__main__.AIOKafkaProducer.send") as mock:

        async def _f() -> None:
            pass

        mock.return_value = asyncio.create_task(_f())

        yield mock

In [ ]:
# | export


async def run_script_and_cancel(
    script: str,
    *,
    script_file: Optional[str] = None,
    cmd: Optional[str] = None,
    cancel_after: int = 10,
    app_name: str = "app",
    kafka_app_name: str = "kafka_app",
    generate_docs: bool = False,
) -> Tuple[int, bytes]:
    """
    Runs a script and cancels it after a predefined time.

    Args:
        script: A python source code to be executed in a separate subprocess.
        script_file: Name of the script where script source will be saved.
        cmd: Command to execute. If None, it will be set to 'python3 -m {Path(script_file).stem}'.
        cancel_after: Number of seconds before sending SIGTERM signal.
        app_name: Name of the app.
        kafka_app_name: Name of the Kafka app.
        generate_docs: Flag indicating whether to generate docs.

    Returns:
        A tuple containing the exit code and combined stdout and stderr as a binary string.
    """
    if script_file is None:
        script_file = "script.py"

    if cmd is None:
        cmd = f"python3 -m {Path(script_file).stem}"

    with TemporaryDirectory() as d:
        consumer_script = Path(d) / script_file

        with open(consumer_script, "w") as file:
            file.write(script)

        if generate_docs:
            logger.info(
                f"Generating docs for: {Path(script_file).stem}:{kafka_app_name}"
            )
            try:
                kafka_app: FastKafka = _import_from_string(
                    f"{Path(script_file).stem}:{kafka_app_name}"
                )
                await asyncer.asyncify(kafka_app.create_docs)()
            except Exception as e:
                logger.warning(
                    f"Generating docs failed for: {Path(script_file).stem}:{kafka_app_name}, ignoring it for now."
                )

        creationflags = 0 if platform.system() != "Windows" else subprocess.CREATE_NEW_PROCESS_GROUP # type: ignore
        proc = subprocess.Popen(
            shlex.split(cmd),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            cwd=d,
            shell=True  # nosec: [B602:subprocess_without_shell_equals_true] subprocess call - check for execution of untrusted input.
            if platform.system() == "Windows"
            else False,
            creationflags=creationflags,
        )
        await asyncio.sleep(cancel_after)
        if platform.system() == "Windows":
            proc.send_signal(signal.CTRL_BREAK_EVENT) # type: ignore
        else:
            proc.terminate()
        output, _ = proc.communicate()

        return (proc.returncode, output)

In [ ]:
# Check exit code 0
script = """
from time import sleep
print("hello")
sleep({t})
"""

exit_code, output = await run_script_and_cancel(script.format(t=0), cancel_after=2)
assert exit_code == 0, f"{exit_code=}, {output=}"
assert output.decode("utf-8").strip() == "hello", output.decode("utf-8")

exit_code, output = await run_script_and_cancel(script.format(t=5), cancel_after=2)
if platform.system() == "Windows":
    assert exit_code == 3221225786, f"{exit_code=}, {output=}"
else:
    assert exit_code < 0, f"{exit_code=}, {output=}"

In [ ]:
# Check exit code 1
script = "exit(1)"

exit_code, output = await run_script_and_cancel(script, cancel_after=1)

assert exit_code == 1
assert output.decode("utf-8") == ""

In [ ]:
# Check exit code 0 and output to stdout and stderr
script = """
import sys
sys.stderr.write("hello from stderr\\n")
sys.stderr.flush()
print("hello, exiting with exit code 0")
exit(0)
"""

exit_code, output = await run_script_and_cancel(script, cancel_after=1)

line_separator = "\r\n" if platform.system() == "Windows" else "\n"

assert exit_code == 0, exit_code
assert (
    output.decode("utf-8") == f"hello from stderr{line_separator}hello, exiting with exit code 0{line_separator}"
), output.decode("utf-8")

In [ ]:
# Check random exit code and output
script = """
print("hello\\nexiting with exit code 143")
exit(143)
"""

exit_code, output = await run_script_and_cancel(script, cancel_after=1)

line_separator = "\r\n" if platform.system() == "Windows" else "\n"

assert exit_code == 143
assert output.decode("utf-8") == f"hello{line_separator}exiting with exit code 143{line_separator}"

print("ok")

ok


In [ ]:
# | export


async def display_docs(docs_path: str, port: int = 4000) -> None:
    """
    Serves the documentation using an HTTP server.

    Args:
        docs_path: Path to the documentation.
        port: Port number for the HTTP server. Defaults to 4000.

    Returns:
        None
    """
    with change_dir(docs_path):
        process = await asyncio.create_subprocess_exec(
            "python3",
            "-m",
            "http.server",
            f"{port}",
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
        )
        try:
            from google.colab.output import eval_js

            proxy = eval_js(f"google.colab.kernel.proxyPort({port})")
            logger.info("Google colab detected! Proxy adjusted.")
        except:
            proxy = f"http://localhost:{port}"
        finally:
            await asyncio.sleep(2)
            display(IFrame(f"{proxy}", 1000, 700))  # type: ignore
            await asyncio.sleep(2)
            await terminate_asyncio_process(process)

In [ ]:
example_html = """


    
        Example
    
    
        This is an example of a simple HTML page with one paragraph.
    

"""

with TemporaryDirectory() as tmp:
    with change_dir(tmp):
        with open(Path(tmp) / "index.html", "w") as index_file:
            index_file.write(example_html)
        await display_docs(docs_path=tmp, port=4000)